<a href="https://colab.research.google.com/github/MatthewYancey/16-9GAN/blob/master/src/process_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing

This notebook takes the video files in a folder and saves the individual frames.

## Imports and Parameters

In [1]:
import os
import glob
import cv2
import shutil
from google.colab import drive
import zipfile

In [3]:
# parameters
drive.mount('/content/gdrive')
DATA_IN_PATH = '/content/gdrive/My Drive/16:9GAN/data_raw/FMA - 16:9/'
DATA_OUT_FRAMES = '/content/frames/'
DATA_OUT_ZIP = '/content/gdrive/My Drive/16:9GAN/data_out/frames_16_9'
os.makedirs(DATA_OUT_FRAMES)

FRAME_SECONDS_SKIP = 0.5
IMAGE_SIZE = 256
SKIP_SECONDS_BEGINNING = 120
SKIP_SECONDS_END = 120
FRAME_LIMIT = None

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Image hash function for checking if we have duplicate images

In [4]:
def dhash(image, hashSize=8):
	# convert the image to grayscale and resize the grayscale image,
	# adding a single column (width) so we can compute the horizontal
	# gradient
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	resized = cv2.resize(gray, (hashSize + 1, hashSize))
	# compute the (relative) horizontal gradient between adjacent
	# column pixels
	diff = resized[:, 1:] > resized[:, :-1]
	# convert the difference image to a hash and return it
	return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

## Video loop and frame saving

In [8]:
files = glob.glob(DATA_IN_PATH + '*')
episode_count = 0

frame_count = 0
for f in files:
    print(f)
    vidcap = cv2.VideoCapture(f)
    video_length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT) / 24)
    success, image = vidcap.read()


    # loops through and save the frames
    image_hashes = []
    while success:
        current_frame = vidcap.get(cv2.CAP_PROP_POS_FRAMES)
        # skips the intro, outtro, and every 24 frames
        if (current_frame >= (SKIP_SECONDS_BEGINNING * 24) and current_frame <= (video_length - SKIP_SECONDS_END) * 24 and current_frame % (FRAME_SECONDS_SKIP * 24) == 0):
            
            # makes a small image and does a hash on it to see if we've had this image before
            image_small = cv2.resize(image, (100, 100))
            image_hash = dhash(image_small)

            # if not a image we've had before we save it 
            if image_hash not in image_hashes:

                image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

                # appends the hash and saves the file
                image_hashes.append(image_hash)
                cv2.imwrite(f'{DATA_OUT_FRAMES}{frame_count}.jpg', image)
                frame_count += 1

        # loop to the next frame
        success, image = vidcap.read()
        
    # breaks the loop if we have enough images
    print(f'Number of images saved: {frame_count}')
    if FRAME_LIMIT != None and frame_count >= FRAME_LIMIT:
        break


/content/gdrive/My Drive/16:9GAN/data_raw/FMA - 16:9/Full Metal Alchemist Brotherhood.E01.Full Metal Alchemist.avi
Number of images saved: 1770
/content/gdrive/My Drive/16:9GAN/data_raw/FMA - 16:9/Full Metal Alchemist Brotherhood.E09.Created Feelings.avi
Number of images saved: 3474
/content/gdrive/My Drive/16:9GAN/data_raw/FMA - 16:9/Full Metal Alchemist Brotherhood.E12.One is All, All is One.avi
Number of images saved: 5288
/content/gdrive/My Drive/16:9GAN/data_raw/FMA - 16:9/Full Metal Alchemist Brotherhood.E10.Separate Destinations.avi
Number of images saved: 6835
/content/gdrive/My Drive/16:9GAN/data_raw/FMA - 16:9/Full Metal Alchemist Brotherhood.E06.Road of Hope.avi
Number of images saved: 8884
/content/gdrive/My Drive/16:9GAN/data_raw/FMA - 16:9/Full Metal Alchemist Brotherhood.E16.Footsteps of a ComradeinArms.avi
Number of images saved: 10532
/content/gdrive/My Drive/16:9GAN/data_raw/FMA - 16:9/Full Metal Alchemist Brotherhood.E11.Miracle at Rush Valley.avi
Number of images sa

In [9]:
img_list = glob.glob(DATA_OUT_FRAMES + '/*')
print('Number of images: %d' % len(img_list))

Number of images: 23393


## Zip frames into a single file

In [10]:
import shutil
shutil.make_archive(DATA_OUT_ZIP, 'zip', DATA_OUT_FRAMES)

'/content/gdrive/My Drive/16:9GAN/data_out/frames_16_9.zip'